# 키움증권 open API+

##### pykiwoom 패키지를 이용하여 KOAStudioSA로부터 주식 API를 받아온다.

증권사에서 배포하는 open API는 32비트 버전의 프로그램을 이용하기 때문에 아나콘다 및 파이썬을 32비트에 해당하는 프로그램으로 맞추어 주어야한다..

pykiwoom 패키지를 install 해주어야한다.

32비트내의 pykiwoom 패키지 안에는 PyQt5 패키지가 존재하지 않는다.  
따라서 Anaconda Prompt를 관리자 권한으로 열어 해당 패키지를 install 한다.

또한 버젼 문제로 인해 PyQt5.13을 PyQt5.12로 다운그레이드하여 준다.

In [1]:
from pykiwoom.kiwoom import *

import time
import pandas as pd
import os

In [2]:
# 로그인하기

kiwoom = Kiwoom()
kiwoom.CommConnect(block = True)   # block = True로 

In [3]:
# 각 시장에 상장된 종목 코드 얻어오기
# option parameter :
#  "0" : 코스피 / "3" : ELW / "4" : 뮤추얼펀드 / "5" : 신주인수권
#  "6" : 리츠 / "8" : ETF  / "9" : 하이얼펀드 / "10" : 코스닥
#  "30" : K-OTC / "50" : 코넥스

kosdaq = kiwoom.GetCodeListByMarket('0')

In [4]:
# 코스피에 상장된 종목코드를 이용하여 종목명 얻기
names ={}
for code in kosdaq:
    names[code] = kiwoom.GetMasterCodeName(code)

In [5]:
names

{'000020': '동화약품',
 '000040': 'KR모터스',
 '000050': '경방',
 '000060': '메리츠화재',
 '000070': '삼양홀딩스',
 '000075': '삼양홀딩스우',
 '000080': '하이트진로',
 '000087': '하이트진로2우B',
 '000100': '유한양행',
 '000105': '유한양행우',
 '000120': 'CJ대한통운',
 '000140': '하이트진로홀딩스',
 '000145': '하이트진로홀딩스우',
 '000150': '두산',
 '000155': '두산우',
 '000157': '두산2우B',
 '000180': '성창기업지주',
 '000210': 'DL',
 '000215': 'DL우',
 '000220': '유유제약',
 '000225': '유유제약1우',
 '000227': '유유제약2우B',
 '000230': '일동홀딩스',
 '000240': '한국앤컴퍼니',
 '000270': '기아',
 '000300': '대유플러스',
 '000320': '노루홀딩스',
 '000325': '노루홀딩스우',
 '000370': '한화손해보험',
 '000390': '삼화페인트',
 '000400': '롯데손해보험',
 '000430': '대원강업',
 '000480': '조선내화',
 '000490': '대동',
 '000500': '가온전선',
 '000520': '삼일제약',
 '000540': '흥국화재',
 '000545': '흥국화재우',
 '000547': '흥국화재2우B',
 '000590': 'CS홀딩스',
 '000640': '동아쏘시오홀딩스',
 '000650': '천일고속',
 '000660': 'SK하이닉스',
 '000670': '영풍',
 '000680': 'LS네트웍스',
 '000700': '유수홀딩스',
 '000720': '현대건설',
 '000725': '현대건설우',
 '000760': '이화산업',
 '000810': '삼성화재',
 '00081

In [6]:
# 삼성전자 코드 가져오기
for k, v in names.items():
    if v == "삼성전자":
        print(k)

005930


In [8]:
# 주식시세 데이터 가져오기

# 멀티데이터의 경우 여러 행으로 데이터가 구성되어있지만 데이터의 행이 많으면 키움증권
# 서버에서는 묶음으로 나누어 보내준다. 이를 연속조회라 한다.
# 따라서 서버로부터 데이터가 남아있는지를 확인한 후 다시 TR을 요청해야한다.

# option : next => 0 : 처음조회 / 2 : 연속조회
# 일별주가
dfs = []
df = kiwoom.block_request("opt10081",
                          종목코드="005930",
                          기준일자="20170703",
                          수정주가구분=1,
                          output="주식일봉차트조회",
                          next=0)
print(df.head())
dfs.append(df)

while kiwoom.tr_remained:
    df = kiwoom.block_request("opt10081",
                          종목코드="005930",
                          기준일자="20170703",
                          수정주가구분=1,
                          output="주식일봉차트조회",
                          next=2)
    dfs.append(df)
    time.sleep(1)
    


# 수집한 데이터를 DataFrame으로 
df = pd.concat(dfs)

# 데이터 내보내기
df.to_excel("samsung.xlsx")

     종목코드      현재가     거래량    거래대금        일자       시가       고가       저가  \
0  005930  2361000  136111  321964  20170703  2375000  2389000  2356000   
1          2377000  237551  562979  20170630  2375000  2381000  2355000   
2          2397000  166131  399659  20170629  2402000  2416000  2397000   
3          2385000  191450  456952  20170628  2380000  2400000  2378000   
4          2415000  192335  463577  20170627  2411000  2420000  2395000   

  수정주가구분  수정비율 대업종구분 소업종구분 종목정보 수정주가이벤트 전일종가  
0                                             
1                                             
2      4  0.00                                
3                                             
4                                             


In [9]:
## 날짜별 주가 데이터 만들기
## 위의 전 종목의 일봉데이터 종가의 데이터를 뽑아온다.

flist = os.listdir()
xlsx_list = [x for x in flist if x.endswith('.xlsx')]
close_data = []

for xls in xlsx_list:
    code = xls.split('.')[0]
    df = pd.read_excel(xls)
    df2 = df[['일자', '현재가']].copy()
    df2.rename(columns={'현재가': code}, inplace=True)
    df2 = df2.set_index('일자')
    df2 = df2[::-1]
    close_data.append(df2)

# concat
df = pd.concat(close_data, axis=1)


In [10]:
df.head(10)

,samsung
일자,
19850104,6472
19850105,6426
19850107,6441
19850108,6357
19850109,6135
19850110,6204
19850111,6242
19850112,6349
19850114,6319


In [9]:
?kiwoom.block_request